In [ ]:
from pathlib import Path 
import geopandas as gpd
import pandas as pd
import os
import datetime
import modules.order_utils as util

#### Create query 

#### Load big datasets 

In [ ]:
FILEPATH = Path(r'vector/RapidEye_Tiles.shp')
df_tiles = gpd.read_file(FILEPATH)

In [ ]:
# load list of existing datasets
DATA_DIR = Path(r'Q:\p_aicore_pf\initze\data\planet\planet_data_inference_grid\tiles')
df_existing = util.get_existing_files(DATA_DIR)
existing_datasets = df_existing.dataset_id.unique()

#### Settings

In [ ]:
#TILE_ID = 5771415#5671617#
TILE_ID = 870513
PROCESSING_DIR = Path(r'C:\Users\initze\OneDrive\100_AI-CORE\05_planet_order\AOIS')
YEAR = '2021'
START = f'{YEAR}-06-01'
END = f'{YEAR}-09-15'
ITEM = 'PSOrthoTile'
ASSET = 'analytic_sr_udm2'
ASSET_QUERY = 'analytic_sr'
CLOUD_MIN = 90
CSV_FILE = PROCESSING_DIR / f'{TILE_ID}.csv'
CSV_FILE_ORDER = PROCESSING_DIR / f'{TILE_ID}_order.csv'

#### Create AOI

In [ ]:
# filter to tile_id and save a geojson
filtered = df_tiles.query(f'tile_id in ({TILE_ID},)')
vector_file = PROCESSING_DIR / f'{TILE_ID}.geojson'
filtered.to_file(vector_file)

#### Run Query for data

In [ ]:
#setup porder query
s = f'porder idlist --input {vector_file} --start "{START}" --end "{END}" --item "{ITEM}" --asset "{ASSET_QUERY}" --filters range:clear_percent:{CLOUD_MIN}:100 --outfile {CSV_FILE}'

# run porder query
print(s)
os.system(s)

#### Filter for relevant datasets
* selected tile_id
* non existing

In [ ]:
try:
    # open csv and filter
    df = pd.read_csv(CSV_FILE, header=None).rename(columns={0:'dataset_id'})
    # filter by tile_id (original query takes neighboring tiles as well 
    df['tile_id'] = df.apply(lambda x:x[0].split('_')[1], axis=1)
except:
    print('No data')

# filter by tile_id
df_filtered = df.query(f'tile_id in ("{TILE_ID:07}",)')
# filter by existing datasets
df_filtered2 = df_filtered[~df_filtered['dataset_id'].isin(existing_datasets)]

print(len(df), len(df_filtered), len(df_filtered2))

df_filtered2

####  Save to order csv

In [ ]:
# save to order csv
output = df_filtered2.iloc[:]['dataset_id']
output.to_csv(CSV_FILE_ORDER, header=None, index=None)

#### Setup Order
* test --op zip and --op zipall
* **zipall has same structure as QGIS**

In [ ]:
s_idcheck = f'porder idcheck --idlist {CSV_FILE_ORDER} --item {ITEM} --asset {ASSET_QUERY}'
s_order = f'porder order --name {TILE_ID}_{START}_{END} --idlist {CSV_FILE_ORDER} --item {ITEM} --bundle {ASSET} --op zipall'

print('Order:', s_order)
print('\nID check:', s_idcheck)

#### Kick-Off Order 

In [ ]:
os.system(s_order)

### Show Orders 

In [ ]:
today = datetime.datetime.now().strftime('%Y-%m-%d')
start = (datetime.datetime.now() - datetime.timedelta(10)).strftime('%Y-%m-%d')
running = !porder ostate --state running --start "{start}" --end "{today}"
util.show_orders(running)

In [ ]:
!pobatch quota